In [1]:
# Step 1: Install Dependencies (this may take about 2-3 min)

# !pip install -q git+https://github.com/openai/whisper.git
# !pip install yt-dlp

import os, re
import torch
from pathlib import Path
import whisper
from whisper.utils import get_writer
from yt_dlp import YoutubeDL


## WhisperAI STT

In [8]:
from pathlib import Path
import torch
import whisper
from whisper.utils import get_writer

# Transcribe the audio data with Whisper
def transcribe_audio(audio_dir="./contents", output_dir="./outputs", output_formats=["txt", "srt"], 
                     mode="youtube", whisper_model = "base", if_Colab=False):
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = whisper.load_model(whisper_model).to(device)

    # Ensure the audio_dir and output_dir exist, create them if they don't
    audio_dir = Path(audio_dir)
    audio_dir.mkdir(parents=True, exist_ok=True)
    
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    if mode == "youtube":
        YouTube_URL = input("Enter YouTube_URL")  # @param {type:"string"}

        def download_audio_from_youtube(url, file_name=None, out_dir=audio_dir):
            print(f"\n==> Downloading audio with yt-dlp...")
            ydl_opts = {
                'format': 'bestaudio/best',
                'outtmpl': f"{out_dir}/{file_name if file_name else '%(title)s.%(ext)s'}",
                'postprocessors': [{
                    'key': 'FFmpegExtractAudio',
                    'preferredcodec': 'mp3',
                    'preferredquality': '192',
                }],
            }
            with YoutubeDL(ydl_opts) as ydl:
                ydl.download([url])
                info_dict = ydl.extract_info(url, download=False)
                file_path = ydl.prepare_filename(info_dict)
                mp3_file_path = Path(file_path).with_suffix('.mp3')
            
            print(f"File downloaded to {mp3_file_path}!")
            return str(mp3_file_path)

        audio = download_audio_from_youtube(YouTube_URL)
        print("\n=======================")
        print(f"\n🔗 YouTube URL: {YouTube_URL}")
        print(f"\n🤖 Whisper Model: {whisper_model}")
        print("\n=======================")
        
    elif mode == "local":
        mp3_files = list(audio_dir.glob("*.mp3"))
        
        if len(mp3_files) == 0:
            raise FileNotFoundError(f"No .mp3 files found in the directory: {output_dir}")
        elif len(mp3_files) > 1:
            raise FileExistsError(f"Multiple .mp3 files found in the directory: {output_dir}. Please specify the file to use.")
        
        audio = mp3_files[0]
        print("\n=======================")
        print(f"\n🔗 Audio Path: {audio}")
        print(f"\n🤖 Whisper Model: {whisper_model}")
        print("\n=======================")
    else:
        raise ValueError("Invalid mode. Please select either 'youtube' or 'local'.")
    
    file_path = Path(audio)  # Convert to Path object

    print(f"\n==> Transcribing audio")
    result = model.transcribe(str(file_path), verbose=False)

    # Save the transcription in the requested formats
    for format in output_formats:
        print(f"\n==> Creating .{format} file")
        writer = get_writer(format, output_dir)
        writer(result, str(file_path.stem))
        
        output_file_path = output_dir / f"{file_path.stem}.{format}"
        if if_Colab:
            from google.colab import files
            files.download(str(output_file_path))
        else:
            print(f"Transcription saved as .{format} file at: {output_file_path}")

    print("\n✨ All Done!")
    print("=======================")
    return result


### 로컬 예제

In [9]:
result = transcribe_audio(mode="local", output_formats=["txt"])



🔗 Audio Path: contents\sample_sound.mp3

🤖 Whisper Model: base


==> Transcribing audio
Detected language: Korean


100%|██████████| 224/224 [00:00<00:00, 1364.62frames/s]


==> Creating .txt file
Transcription saved as .txt file at: outputs\sample_sound.txt

✨ All Done!


## Youtube 예제

In [10]:
result = transcribe_audio(mode="youtube", output_formats=["txt"])


==> Downloading audio with yt-dlp...
[youtube] Extracting URL: https://youtu.be/GEYxeMYMtE4
[youtube] GEYxeMYMtE4: Downloading webpage
[youtube] GEYxeMYMtE4: Downloading ios player API JSON
[youtube] GEYxeMYMtE4: Downloading web creator player API JSON
[youtube] GEYxeMYMtE4: Downloading player 57c75fa4
[youtube] GEYxeMYMtE4: Downloading m3u8 information
[info] GEYxeMYMtE4: Downloading 1 format(s): 251
[download] Destination: outputs\[시계열] Ch1. 데이터와 인공지능, 그리고 데이터 분석이란？.webm
[download] 100% of   16.03MiB in 00:00:06 at 2.43MiB/s   
[ExtractAudio] Destination: outputs\[시계열] Ch1. 데이터와 인공지능, 그리고 데이터 분석이란？.mp3
Deleting original file outputs\[시계열] Ch1. 데이터와 인공지능, 그리고 데이터 분석이란？.webm (pass -k to keep)
[youtube] Extracting URL: https://youtu.be/GEYxeMYMtE4
[youtube] GEYxeMYMtE4: Downloading webpage
[youtube] GEYxeMYMtE4: Downloading ios player API JSON
[youtube] GEYxeMYMtE4: Downloading web creator player API JSON
[youtube] GEYxeMYMtE4: Downloading m3u8 information
File downloaded to outputs\[시

100%|██████████| 125648/125648 [01:19<00:00, 1570.74frames/s]


==> Creating .txt file
Transcription saved as .txt file at: outputs\[시계열] Ch1. 데이터와 인공지능, 그리고 데이터 분석이란？.txt

✨ All Done!
